In [1]:
with open("input23.txt", "r") as f:
    data = f.read().strip().split("\n")

In [2]:
import re
import math
import numpy as np

In [3]:
arr = np.array([list(map(int, re.findall(r'-?\d+', d))) for d in data])

In [4]:
radius_index = np.argmax(arr[:,3])
dists = arr[:,:3] - arr[radius_index][:3]
matches = (np.sum(np.abs(dists),axis=1)<=arr[radius_index,3]).sum()
print(matches)

652


In [5]:
def get_box_count(arr,x0,y0,z0,x1,y1,z1):
    """
    Get the count of unique nanobots which can reach this cube
    
    For each of the x,y,z values of a nanobot - 
    If the value is in between x0 (or y0, etc) and x1, it's 0 steps to reach this edge
    Otherwise the number of steps will be min of steps to reach x0 or x1
    Add up all the shortest steps for x,y,z for each nanobot, and compare to the nanobot's radius, to get count
    """
    lows = [x0,y0,z0]
    highs = [x1,y1,z1]
    
    dist = arr[:,:3]
    
    #Save this for later - values which are in between bounds and should be set to 0
    zero_mask = (dist>=lows) & (dist<=highs)
    
    diff1 = np.abs(dist-lows)
    diff2 = np.abs(dist-highs)
    diff = np.minimum(diff1,diff2)
    
    #Apply the mask, and then compare the fewest steps to reach this box to our radius
    diff[zero_mask]=0
    fewest_steps = np.sum(diff,axis=1)
    return (fewest_steps<=arr[:,3]).sum()

In [6]:
def get_point_count(arr,x,y,z):
    """Number of nanobots in range of point"""
    res = np.abs(arr[:,:3]-[x,y,z])
    return (res.sum(axis=1)<=arr[:,3]).sum()

In [7]:
def get_start_coords(arr):
    """Get initial bounding box that should cover entire area, and be easy to subdivide"""
    x0,x1 = min(arr[:,0]), max(arr[:,0])
    y0,y1 = min(arr[:,1]), max(arr[:,1])
    z0,z1 = min(arr[:,2]), max(arr[:,2])
    range_ = max((x1-x0,y1-y0,z1-z0))

    #Be sure it's evenly divisible by 3
    range_ = 3**math.ceil(math.log(range_,3))
    return x0,y0,z0, x0+range_,y0+range_,z0+range_

In [8]:
def divide_box(x0,y0,z0, x1,y1,z1):
    """Subdivide into 27 equal sized partitions"""
    step = (x1-x0)//3
    boxes = []
    for x in range(x0,x1,step):
        for y in range(y0,y1,step):
            for z in range(z0,z1,step):
                if step>1:
                    boxes.append((x,y,z, x+step,y+step, z+step))
                else:
                    boxes.append((x,y,z, x,y,z))
    return boxes             

In [9]:
#Value to beat - start at 0,0,0
best_score = 0
best_matches = get_point_count(arr, 0,0,0)

#Get starting coordinates - ensure nicely subdivisible cube
start_coords = get_start_coords(arr)

box_matches = get_box_count(arr,*start_coords)

box_stack = [(box_matches, sum([abs(x) for x in start_coords[:3]]), start_coords)]

while box_stack:
    #Want it sorted by - matches and then coordinate sum (lowest to highest, as we'll pop from back)
    box_stack.sort()
    
    #Repeatedly pop the bounding box which contains the most reachable points
    curr_box = box_stack.pop()
    
    box_matches, box_score, coords = curr_box
    #Since we're sorting each time, once the best case scenario would leave us worse off, we can halt
    if box_matches < best_matches or (box_matches==best_matches and box_score>best_score):
        break
    
    #Divide box into 27 partitions
    divided = divide_box(*coords)
    for d in divided:
        #d is coordinates of a box (or a point) like: x0,y0,z0, x1,y1,z1
        
        #If it's a point
        if d[:3]==d[3:]:            
            matches = get_point_count(arr, *d[:3])
            score = sum([abs(x) for x in d[:3]])
            if matches>best_matches or (matches==best_matches and score<best_score):
                best_score = score
                best_matches = matches
        #Otherwise it's a box - subdivide and add all potential best score areas to our stack
        else:
            box_matches = get_box_count(arr, *d)
            box_best_score = sum([abs(x) for x in d[:3]])
            if box_matches>best_matches or (box_matches==best_matches and box_best_score>best_score):
                box_stack.append((box_matches, box_best_score, d))
print(best_score)

164960498
